###Generating LibriMixDataset

In [1]:
!git clone https://github.com/JorisCos/LibriMix

Cloning into 'LibriMix'...
remote: Enumerating objects: 274, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 274 (delta 71), reused 58 (delta 58), pack-reused 163
Receiving objects: 100% (274/274), 33.43 MiB | 21.02 MiB/s, done.
Resolving deltas: 100% (116/116), done.


In [2]:
%cd LibriMix

/content/LibriMix


In [3]:
!mkdir data

In [4]:
!ls

data  generate_librimix.sh  LICENSE  metadata  README.md  requirements.txt  scripts


In [5]:
!apt-get install sox libsox-fmt-all

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-alsa
  libsox-fmt-ao libsox-fmt-base libsox-fmt-mp3 libsox-fmt-oss libsox-fmt-pulse libsox3 libwavpack1
Suggested packages:
  libaudio2 libsndio6.1
The following NEW packages will be installed:
  libao-common libao4 libid3tag0 libmad0 libopencore-amrnb0 libopencore-amrwb0 libsox-fmt-all
  libsox-fmt-alsa libsox-fmt-ao libsox-fmt-base libsox-fmt-mp3 libsox-fmt-oss libsox-fmt-pulse
  libsox3 libwavpack1 sox
0 upgraded, 16 newly installed, 0 to remove and 39 not upgraded.
Need to get 800 kB of archives.
After this operation, 2,533 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libao-common all 1.2.2+20180113-1.1ubuntu3 [6,568 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libao4 amd64 1.2.2+

In [6]:
!chmod +x /content/LibriMix/generate_librimix.sh

In [7]:
!pip install pysndfx
!pip install soundfile

  Preparing metadata (setup.py) ... done
  Created wheel for pysndfx: filename=pysndfx-0.3.6-py3-none-any.whl size=6539 sha256=1821799a451202012c9a5509eed85390cf12162b6e1c43228ee0a67f75f7e707
  Stored in directory: /root/.cache/pip/wheels/e8/73/04/275af1c6a789bbc12f388bce2d5353e77579e3aaadd36c61fe
Successfully built pysndfx


In [13]:
!rm -r /content/LibriMix/data/*

In [8]:
!/content/LibriMix/generate_librimix.sh data

Download wham_noise into data
Download LibriSpeech/test-clean into data
--2024-03-27 17:27:19--  https://my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com/wham_noise.zip
--2024-03-27 17:27:19--  http://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... Resolving my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com (my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com)... 3.5.20.177, 3.5.25.30, 54.231.133.9, ...
Connecting to my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com (my-bucket-a8b4b49c25c811ee9a7e8bba05fa24c7.s3.amazonaws.com)|3.5.20.177|:443... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:80... connected.
connected.
HTTP request sent, awaiting response... HTTP request sent, awaiting response... 302 Found
Location: http://openslr.elda.org/resources/12/test-clean.tar.gz [following]
--2024-03-27 17:27:19--  http://openslr.elda.org/resources/12/test-clean.tar.gz
Resolving o

In [10]:
!pwd

/content/LibriMix


###Loading the LibriMix dataset

In [ ]:
!pip install torchmetrics

In [ ]:
!pip install speechbrain

In [13]:
# import os
# import random
# from speechbrain.inference.separation import SepformerSeparation as separator
# import torchaudio
# import torch
# from torchmetrics.audio import ScaleInvariantSignalNoiseRatio, ScaleInvariantSignalDistortionRatio

In [12]:
!rm -r /content/sample_data/*

In [23]:
# Load dataset as a tuple (mix, s1, s2) (only paths)
def load_dataset(mix_folder, s1_folder, s2_folder):
    mix_files = os.listdir(mix_folder)
    dataset = []
    for mix_file in mix_files:
        mix_path = os.path.join(mix_folder, mix_file)
        s1_path = os.path.join(s1_folder, mix_file)
        s2_path = os.path.join(s2_folder, mix_file)
        dataset.append((mix_path, s1_path, s2_path))
    return dataset

# Perform 70:30 split
def train_test_split(dataset, split_ratio=0.7):
    random.shuffle(dataset)
    split_idx = int(len(dataset) * split_ratio)
    train_set = dataset[:split_idx]
    test_set = dataset[split_idx:]
    return train_set, test_set



# Load the dataset
mix_folder = "/content/LibriMix/data/Libri2Mix/wav16k/min/test/mix_both"
s1_folder = "/content/LibriMix/data/Libri2Mix/wav16k/min/test/s1"
s2_folder = "/content/LibriMix/data/Libri2Mix/wav16k/min/test/s2"
dataset = load_dataset(mix_folder, s1_folder, s2_folder)

# Split the dataset
train_set, test_set = train_test_split(dataset, split_ratio=0.7)


###Evaluate performance using SepFormer Model

In [ ]:
# Load the model
model = separator.from_hparams(source="speechbrain/sepformer-whamr", savedir='pretrained_models/sepformer-whamr', run_opts={"device":"cuda"})

In [24]:
# Perform inference using the provided script
def perform_inference(model, test_set):
    si_snr_metric = ScaleInvariantSignalNoiseRatio()
    si_sdr_metric = ScaleInvariantSignalDistortionRatio()
    si_snr_values = []
    si_sdr_values = []
    for mix_path, s1_path, s2_path in test_set:
        est_sources = model.separate_file(path=mix_path)
        s1, _ = torchaudio.load(s1_path)
        s2, _ = torchaudio.load(s2_path)

        # Calculate SI-SNR and SI-SDR for original sources
        original_sources = torch.stack((s1, s2))
        si_snr_orig = si_snr_metric(original_sources, original_sources)
        si_sdr_orig = si_sdr_metric(original_sources, original_sources)

        # Calculate SI-SNR and SI-SDR for estimated sources
        si_snr_est = si_snr_metric(est_sources, original_sources)
        si_sdr_est = si_sdr_metric(est_sources, original_sources)

        # Calculate SI-SNR and SI-SDR improvement
        si_snr_improvement = si_snr_est - si_snr_orig
        si_sdr_improvement = si_sdr_est - si_sdr_orig

        # Append metric values to lists
        si_snr_values.append(si_snr_improvement.item())
        si_sdr_values.append(si_sdr_improvement.item())

    # Calculate average values
    avg_si_snr = sum(si_snr_values) / len(si_snr_values)
    avg_si_sdr = sum(si_sdr_values) / len(si_sdr_values)

    # Log average values
    print(f"Average SI-SNR Improvement: {avg_si_snr}")
    print(f"Average SI-SDR Improvement: {avg_si_sdr}")

    # Optionally, you can also return the average values if needed
    return avg_si_snr, avg_si_sdr


hyperparams.yaml:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

masknet.ckpt:   0%|          | 0.00/113M [00:00<?, ?B/s]

encoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

decoder.ckpt:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# Perform inference on the test set and calculate metrics
avg_si_snri, avg_si_sdri = perform_inference(model, test_set)

In [ ]:
print(avg_si_snri)

In [ ]:
print(avg_si_sdri)

In [ ]:
# from speechbrain.inference.separation import SepformerSeparation as separator
# import torchaudio

# model = separator.from_hparams(source="speechbrain/sepformer-whamr", savedir='pretrained_models/sepformer-whamr', run_opts={"device":"cuda"})

# # for custom file, change path
# est_sources = model.separate_file(path='speechbrain/sepformer-wsj02mix/test_mixture.wav')

# torchaudio.save("source1hat.wav", est_sources[:, :, 0].detach().cpu(), 8000)
# torchaudio.save("source2hat.wav", est_sources[:, :, 1].detach().cpu(), 8000)


###Fine tune the SepFormer on train split

In [ ]:
% cd ..
!git clone https://github.com/speechbrain/speechbrain/

In [ ]:
%cd speechbrain
!pip install -r requirements.txt
!pip install -e .

In [ ]:
%cd  recipes/WHAMandWHAMR/separation
!python train.py hparams/sepformer-whamr.yaml --data_folder=data --rir_path=output_data

###Evaluate performance of fine-tuned model on test split

In [ ]:
!python train.py hparams/sepformer-wham.yaml --data_folder data --test_only